In [26]:
import polars as pl
from datetime import date, timedelta
import random
import string

# Function to generate a date range from 2000 to 2024
def generate_date_range(start_year, end_year):
    start_date = date(start_year, 1, 1)
    end_date = date(end_year, 12, 31)
    delta = end_date - start_date
    return [start_date + timedelta(days=i) for i in range(delta.days + 1)]

# Generate a list of dates from 2000 to 2024
dates = generate_date_range(2, 9999)

# Generate random groups, values, and set ctr to 1 for each entry
groups = [random.choice(string.ascii_uppercase) for _ in dates]
values = [random.randint(1, 5000000) for _ in dates]
ctrs = [1 for _ in dates]

# Create the DataFrame
df = pl.DataFrame({
    "group": groups,
    "values": values,
    "date": dates,
    "ctr": ctrs
})

df.describe()

statistic,group,values,date,ctr
str,str,f64,str,f64
"""count""","""3651694""",3.651694e6,"""3651694""",3.651694e6
"""null_count""","""0""",0.0,"""0""",0.0
"""mean""",null,2.4999e6,"""5000-12-31""",1.0
"""std""",null,1.4434e6,null,0.0
"""min""","""A""",1.0,"""0002-01-01""",1.0
"""25%""",null,1.249815e6,"""2501-07-02""",1.0
"""50%""",null,2.500417e6,"""5001-01-01""",1.0
"""75%""",null,3.749325e6,"""7500-07-02""",1.0
"""max""","""Z""",5e6,"""9999-12-31""",1.0


In [27]:
# Define a custom function for rolling sum operation on a DataFrame grouped by 'group'
def rolling_sum(group_df):
    # Selects specific columns and applies a rolling sum on the 'ctr' column with a window of 3
    return group_df.select([
        pl.col("group"),                     # Include the 'group' column
        pl.col("date"),                      # Include the 'date' column
        pl.col("ctr").rolling_sum(window_size="1y",by="date", min_periods=1)  # Apply rolling sum on 'ctr'
    ])

# Group the DataFrame by 'group', apply the custom rolling_sum function, and sort the result
result = df.group_by("group").map_groups(rolling_sum).sort(by=["group","date"], descending=False)

# Display the first 6 rows of the resulting DataFrame to showcase the rolling sum operation
result.head(100)

group,date,ctr
str,date,i64
"""A""",0002-01-16,1
"""A""",0002-02-14,2
"""A""",0002-02-22,3
"""A""",0002-02-23,4
"""A""",0002-03-19,5
"""A""",0002-03-26,6
"""A""",0002-05-25,7
"""A""",0002-06-03,8
"""A""",0002-08-25,9


In [28]:
##trying a lazy frame method of doing this

lazy_df = df.lazy()
#create a query
q = (
    lazy_df
    .with_columns(
        pl.col("ctr").rolling_sum(window_size="1y", min_periods=1, by="date")
        .over("group")
        .alias("group_ctr_rolling")
    )
    .sort(by=["group","date"], descending=False)
    # .with_columns(
    #     pl.col("ctr")
    #     .rolling_sum()
    # )
)

lazy_df = q.collect(streaming=True)

lazy_df.head(100)

group,values,date,ctr,group_ctr_rolling
str,i64,date,i64,i64
"""A""",4663932,0002-01-16,1,1
"""A""",1665068,0002-02-14,1,2
"""A""",1737526,0002-02-22,1,3
"""A""",1997747,0002-02-23,1,4
"""A""",1652297,0002-03-19,1,5
"""A""",812302,0002-03-26,1,6
"""A""",667878,0002-05-25,1,7
"""A""",1521835,0002-06-03,1,8
"""A""",1815003,0002-08-25,1,9
